# 여성 안전 지수 - 인프라 개선 시급도 분석

이 코드는 서울시 각 지역구('지역구')의 취약성 요인(범죄율, 여성 독거노인 등)과 방어력 요인(CCTV, 경찰서 등)을 종합하여 '인프라 개선 시급도 지수'를 계산하는 것을 목표로 합니다.

## 단계 0: 라이브러리 임포트 및 기본 설정

목표: 데이터 분석 및 시각화에 필요한 라이브러리(pandas, numpy, sklearn, scipy, matplotlib, seaborn, folium 등)를 불러오고, 지역구 이름 통일을 위한 함수를 정의합니다. 

시각화를 위한 한글 폰트 설정도 이 단계에서 수행합니다.

In [12]:
# [SORA Project] 
# ---------------------------------------------------
# 0단계: 라이브러리 임포트 및 기본 설정
# ---------------------------------------------------
print("-" * 30)
print("--- 0단계: 라이브러리 임포트 및 기본 설정 시작 ---")

# --- 0.1: 필수 라이브러리 임포트 ---
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm
from sklearn.preprocessing import MinMaxScaler
from scipy import stats
import folium
import json
from folium.features import DivIcon

# --- 0.2: 경로 설정 ---
# 이 노트북(.ipynb)이 'data/src' 폴더에 있다고 가정합니다.
try:
    # .py 스크립트로 실행 시
    CWD = os.path.dirname(os.path.abspath(__file__))
except NameError:
    # Jupyter Notebook 등 인터랙티브 환경
    CWD = os.getcwd()
    print("[0단계 설정] 경고: 인터랙티브 환경으로 감지됨. CWD를 현재 작업 디렉토리로 설정합니다.")
    if not (os.path.basename(CWD) == 'src' and os.path.basename(os.path.dirname(CWD)) == 'data'):
        print(f"[0단계 설정] 경고: 현재 CWD '{CWD}'가 예상 경로 '.../data/src'와 다를 수 있습니다.")

# [수정] CWD는 절대 경로를 사용 (사용자 요청 형식과 일치시키기 위함)
CWD = os.path.abspath(CWD) 
# [수정] CWD 출력 위치를 1단계로 이동

# 원본 데이터가 있는 폴더 (src 폴더의 상위 폴더 -> data -> preprocessing)
DATA_PATH = os.path.join(CWD, '..', 'data', 'preprocessing')
# 결과물(pkl, png, html)이 저장될 폴더 (src 폴더의 상위 폴더 -> data)
OUTPUT_PATH = os.path.join(CWD, '..', 'data')

# 결과물 저장 폴더 생성 (이미 있으면 넘어감)
os.makedirs(OUTPUT_PATH, exist_ok=True)
print(f"[0단계 설정] 결과물 저장 폴더 확인/생성 완료: {OUTPUT_PATH}")

# --- 0.3: 한글 폰트 설정 함수 ---
def set_korean_font():
    """운영체제에 맞는 한글 폰트를 설정합니다."""
    print("\n--- [폰트 설정 시작] ---")
    try:
        if os.name == 'nt': # Windows
            plt.rc('font', family='Malgun Gothic')
            print("[폰트 설정] 'Malgun Gothic' (Windows) 폰트 설정 완료.")
        elif 'DARWIN' in os.uname().sysname: # macOS
            plt.rc('font', family='AppleGothic')
            print("[폰트 설정] 'AppleGothic' (macOS) 폰트 설정 완료.")
        else: # Linux / Colab
            # Colab 폰트 설치 (필요시 주석 해제)
            # !sudo apt-get install -y fonts-nanum
            # !sudo fc-cache -fv
            # !rm ~/.cache/matplotlib -rf
            plt.rc('font', family='NanumBarunGothic')
            print("[폰트 설정] 'NanumBarunGothic' (Linux/Colab) 폰트 설정 완료.")
    except Exception as e:
        print(f"[폰트 설정] 폰트 설정 중 오류 발생 (무시하고 진행): {e}")

    plt.rcParams['axes.unicode_minus'] = False # 마이너스 부호 깨짐 방지
    print("--- [폰트 설정 완료] ---")

# --- 0.4: Pandas/폰트 설정 실행 ---
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
set_korean_font()

print("--- ✅ 0단계: 라이브러리 임포트 및 기본 설정 완료 ---")
print("-" * 50)


------------------------------
--- 0단계: 라이브러리 임포트 및 기본 설정 시작 ---
[0단계 설정] 경고: 인터랙티브 환경으로 감지됨. CWD를 현재 작업 디렉토리로 설정합니다.
[0단계 설정] 경고: 현재 CWD 'c:\Users\user\Desktop\경진대회\SORA_Project\src'가 예상 경로 '.../data/src'와 다를 수 있습니다.
[0단계 설정] 결과물 저장 폴더 확인/생성 완료: c:\Users\user\Desktop\경진대회\SORA_Project\src\..\data

--- [폰트 설정 시작] ---
[폰트 설정] 'Malgun Gothic' (Windows) 폰트 설정 완료.
--- [폰트 설정 완료] ---
--- ✅ 0단계: 라이브러리 임포트 및 기본 설정 완료 ---
--------------------------------------------------


## 1단계: 2023년 기준 데이터 로드 및 병합

분석 기준 시점인 '2023년' 데이터를 중심으로 [취약성 X], [방어력 Y], [검증용] 데이터를 모두 불러와 '지역구'를 기준으로 하나의 마스터 테이블(DataFrame)로 병합

주요 변수:

취약성(X): Sexual_Crime_Count(성범죄건수), Sexual_Assault_Risk_Index(성폭력위험지수), Elderly_Women_Alone(여성독거노인)

방어력(Y): CCTV, Safety_Bell(안심벨), Police_Station(지구대/파출소), Police_Center(치안센터), Police_Officers(경찰관수)

참고: Pubs(유흥주점), Calls_112(112신고), Total_Crime_5(5대범죄)

** '참고' 변수인 이유

1) Total_Crime_5 (5대범죄)의 역할: 최종 결과 지표 (Target Outcome)
Total_Crime_5는 우리가 최종적으로 예측하거나 개선 효과를 측정하려는 '목표(Goal)'입니다. 즉, 시설 지수(CCTV, 안심벨 등)가 높아졌을 때 감소하기를 기대하는 종속 변수(Dependent Variable)입니다.
'시설 지수'를 만드는데 5대 범죄를 직접 취약점으로 사용하면 지수가 '자신의 결과를 설명'하게 되어 분석의 객관성을 잃을 수 있습니다. 따라서 이 변수는 산출된 지수를 검증하고 해석하는 '참고' 지표로 사용됩니다.

2) Calls_112 (112신고)의 역할: 활동 및 인프라 지표 (Activity & Infrastructure)
Calls_112는 실제 범죄 건수라기보다는 경찰의 활동량이나 주민의 치안 수요/인지도를 나타내는 지표입니다.

112 신고에는 실제 범죄 외에도 소음, 주취자, 위치 문의 등 비범죄성 신고가 상당수 포함됩니다.

이를 직접적인 '취약점'으로 사용할 경우, 치안 서비스가 잘 갖춰져서 신고율이 높은 지역이 오히려 '더 위험한 지역'으로 오해되는 결과를 낳을 수 있습니다 (높은 신고율이 높은 범죄율을 의미하지 않을 수 있음).
따라서 지역별 치안 환경의 특성을 이해하거나, 경찰력(경찰관 인원수, 지구대 수) 투입의 적정성을 판단하는 '참고' 변수로 활용됩니다.

요약: 시설 지수는 안전 시설물(CCTV, 안심벨 등)을 중심으로 구성하며, 5대 범죄는 이 지수의 효과를 검증하는 목표 변수로, 나머지 두 변수는 지수의 해석에 도움을 주는 상황적 '참고' 변수로 활용하는 것이 분석의 목적에 부합합니다.

In [14]:
# ---------------------------------------------------
# 1단계: 2023년 기준 데이터 로드 및 병합
# ---------------------------------------------------

# --- [수정] 1단계 시작과 동시에 요청하신 경로 출력 ---
print("--- 1. 데이터 로드 및 전처리 시작 ---")
print(f"현재 작업 디렉토리(CWD): {CWD}")
print(f"데이터를 읽을 폴더 경로: {DATA_PATH}")
# -------------------------------------------------

def read_csv_with_fallback(file_path, **kwargs):
    """CSV 파일을 'cp949'로 먼저 읽고, 실패 시 'utf-8'로 다시 시도합니다."""
    try:
        return pd.read_csv(file_path, encoding='cp949', **kwargs)
    except UnicodeDecodeError:
        return pd.read_csv(file_path, encoding='utf-8', **kwargs)
    except Exception as e:
        print(f"!!! '{os.path.basename(file_path)}' 파일 읽기 중 오류 발생: {e}")
        raise

try:
    # --- 1.1: [취약성 X] 변수 로드 ---
    # [수정] 사용자님의 1단계 스크립트 기준 변수명 사용
    df_target_crime = read_csv_with_fallback(os.path.join(DATA_PATH, 'X03_전처리_범죄발생_건수_20231231기준.csv'), usecols=['지역구', '합계']).rename(columns={'합계': 'Sexual_Crime_Count'})
    df_risk_index = read_csv_with_fallback(os.path.join(DATA_PATH, 'X07_전처리_성폭력위험요소_지수.csv'), usecols=['여성_잠재적_위험_지수']).rename(columns={'여성_잠재적_위험_지수': 'Sexual_Assault_Risk_Index'})
    base_df = read_csv_with_fallback(os.path.join(DATA_PATH, 'X09_전처리_유흥주점_수.csv'), usecols=['지역구'])
    df_pubs = read_csv_with_fallback(os.path.join(DATA_PATH, 'X09_전처리_유흥주점_수.csv')).rename(columns={'지역구별 주점 수': 'Pubs'})
    df_elderly = read_csv_with_fallback(os.path.join(DATA_PATH, 'X13_전처리_여성독거노인_현황_2020_2024.csv'), usecols=['지역구', '2024']).rename(columns={'2024': 'Elderly_Women_Alone'})
    
    df_risk_index['지역구'] = base_df['지역구'] # X07에 지역구 부여

    # --- 1.2: [방어력 Y] 변수 로드 ---
    # [수정] 사용자님의 1단계 스크립트 기준 (치안센터 제외)
    df_cctv = read_csv_with_fallback(os.path.join(DATA_PATH, 'Y06_전처리_CCTV_설치수_20230421기준.csv'), usecols=['지역구', 'CCTV 설치수']).rename(columns={'CCTV 설치수': 'CCTV'})
    df_bell = read_csv_with_fallback(os.path.join(DATA_PATH, 'Y06_전처리_안심벨_설치수_20230421기준.csv'), usecols=['지역구', '안심벨 설치수']).rename(columns={'안심벨 설치수': 'Safety_Bell'})
    df_station = read_csv_with_fallback(os.path.join(DATA_PATH, 'Y14_전처리_지구대파출소_설치수_20241231기준.csv')).rename(columns={'지역구별 지구대/파출소 수': 'Police_Station'})

    # --- 1.3: [검증 V] 및 [참고 Ref] 변수 로드 ---
    # [수정] 사용자님의 1단계 스크립트 기준 변수명 사용
    df_total_crime = read_csv_with_fallback(os.path.join(DATA_PATH, 'X01_전처리_5대범죄_발생검거_누적건수_2023.csv'), usecols=['지역구', '총_누적_발생_건수', '장기_누적_검거율(%)'])
    df_total_crime = df_total_crime.rename(columns={'총_누적_발생_건수': 'Total_Crime_5', '장기_누적_검거율(%)': 'Arrest_Rate_5'})
    df_calls = read_csv_with_fallback(os.path.join(DATA_PATH, 'X02_전처리_112신고출동_연간건수_2020_2024.csv'), usecols=['지역구', '2023']).rename(columns={'2023': 'Calls_112'})
    
    # --- 1.4: 모든 DataFrame 병합 ---
    # [수정] 사용자님의 1단계 스크립트 기준 (how='outer')
    df = base_df
    all_dfs = [
        df_target_crime, df_risk_index, df_elderly, df_pubs, # (취약성 X)
        df_cctv, df_bell, df_station,                       # (방어력 Y)
        df_total_crime, df_calls                            # (검증 V, 참고 Ref)
    ]

    for temp_df in all_dfs:
        df = pd.merge(df, temp_df, on='지역구', how='outer')

    # --- 1.5: 후처리 ---
    df = df.fillna(0)
    df = df.set_index('지역구')
    
    # '남대문구' 등 25개 자치구가 아닌 행 제거
    if '남대문구' in df.index:
        df = df.drop('남대문구', errors='ignore') 
        
    print("--- ✅ 1. 데이터 로드 및 전처리 완료 ---")

    # --- [1단계 중간 결과 출력] ---
    print("\n[1단계 중간 결과: 병합된 데이터 (상위 5개)]")
    print(df.head())
    print("\n[1단계 중간 결과: 데이터 정보]")
    df.info()
    
    # --- [1단계: 중간 저장] ---
    # 2단계로 넘기기 위해 pkl 파일로 저장
    output_pkl_path = os.path.join(OUTPUT_PATH, 'data_01_merged.pkl')
    df.to_pickle(output_pkl_path)
    print(f"\n[1단계] 중간 병합 데이터가 '{output_pkl_path}' 파일로 저장되었습니다.")
    
except FileNotFoundError as e:
    print(f"!!! 오류: '{e.filename}' 파일을 찾을 수 없습니다.")
    print(f"!!! 탐색 경로: '{DATA_PATH}'") 
    print("!!! 노트북이 'data/src'에 있고 데이터가 'data/preprocessing'에 있는지 확인하세요.")
except Exception as e:
    print(f"!!! 1단계 데이터 로드 중 오류 발생: {e}")

print("-" * 50)


--- 1. 데이터 로드 및 전처리 시작 ---
현재 작업 디렉토리(CWD): c:\Users\user\Desktop\경진대회\SORA_Project\src
데이터를 읽을 폴더 경로: c:\Users\user\Desktop\경진대회\SORA_Project\src\..\data\preprocessing
--- ✅ 1. 데이터 로드 및 전처리 완료 ---

[1단계 중간 결과: 병합된 데이터 (상위 5개)]
     Sexual_Crime_Count  Sexual_Assault_Risk_Index  Elderly_Women_Alone   Pubs   CCTV  Safety_Bell  Police_Station  Total_Crime_5  Arrest_Rate_5  Calls_112
지역구                                                                                                                                                        
강남구              1099.0                     3902.0              16359.0  165.0  122.0         89.0            14.0         6763.0          73.81   120466.0
강동구               294.0                     1103.0              14763.0   98.0   48.0         46.0             9.0         3398.0          73.43    91280.0
강북구               261.0                     1201.0              14192.0   69.0   92.0         77.0             9.0         2497.0          85.78    

## 2단계: 취약성(X) 및 방어력(Y) 점수 계산

목표 : 1단계에서 병합한 원본 데이터를 사용해, 모델의 핵심인 '종합 취약성(X) 점수'와 '종합 방어력(Y) 점수'를 계산하고 시각화합니다.

1) 각 변수를 0~1 사이 값으로 정규화(Min-Max Scaling)합니다.

- 취약성(X): Elderly_Women_Alone, Sexual_Crime_Count, Sexual_Assault_Risk_Index

- 방어력(Y): CCTV, Safety_Bell, Police_Station, Police_Center, Police_Officers

2) 정규화된 취약성 변수들의 평균을 내어 '종합 취약성(X) 점수'를 계산합니다.

   정규화된 방어력 변수들의 평균을 내어 '종합 방어력(Y) 점수'를 계산합니다.

3) 두 점수를 지역구별로 비교하는 막대 차트를 저장하고 그래프를 해석합니다.

## 3단계: '개선 시급도 지수' 산출 (원시 점수)

목표 : 가설(시급도 = 취약성 - 방어력)에 따라, 2단계에서 계산한 두 종합 점수를 빼서 '개선 시급도 지수'의 원시 점수(Urgency_Index)를 계산하고 시각화합니다.
- Urgency_Index = Vulnerability_Score - Defense_Score 공식을 적용합니다.

  (값이 클수록 시급함, 양수 = 취약 > 방어, 음수 = 방어 > 취약)

## 4단계: '최종 시급도 지수' 산출 (0~100점)

목표 : 3단계에서 계산한 원시 시급도 지수(예: -0.3 ~ +0.2)는 직관적으로 이해하기 어렵습니다. 따라서 이 점수를 0~100점 척도로 변환하여 '최종 시급도 지수'(Urgency_Index_Scaled)를 만들고 시각화합니다.
Min-Max 스케일러의 범위를 (0, 100)으로 설정합니다.

- Urgency_Index를 이 스케일러에 적용하여 '최종 시급도 지수'를 계산합니다.

    (가장 시급한 지역 = 100점, 가장 양호한 지역 = 0점)

- 차트를 해석하여 어떤 지역구가 개선 우선순위가 높은지 설명합니다.

## 5단계: 모델 내부 검증 (상관관계 분석)

목표 : 우리가 2단계에서 '취약성'과 '방어력'으로 묶은 변수들이 통계적으로도 타당한지 [내부 검증]을 수행합니다. (EDA: 탐색적 데이터 분석)
* (p < 0.05), ** (p < 0.01)의 의미를 주석에 명시합니다.

1) stats.pearsonr 함수를 사용해 모든 변수([취약성X], [방어력Y], 종합점수) 간의 상관계수와 p-value를 계산합니다.

2) 표와 히트맵을 해석하여, 우리가 가정한 변수 그룹(취약성/방어력) 내의 관계가 실제로 강한지, 그룹 간의 관계는 어떤지 설명합니다.

## 6단계: 모델 외부 검증 (타당성 검증)